In [1]:
import numpy as np
import pandas as pd
from risk_data import get_factor_data

In [2]:
factor_data = get_factor_data()

c:\Users\bkrai\Source\venv-bklm\Lib\site-packages\zarr\codecs\vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
c:\Users\bkrai\Source\venv-bklm\Lib\site-packages\zarr\codecs\vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)
c:\Users\bkrai\Source\venv-bklm\Lib\site-packages\zarr\codecs\vlen_utf8.py:44: UserWarning: The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  return cls(**configuration_parsed)


In [18]:
vix = factor_data.cret.sel(factor_name='^VIX3M').to_pandas().dropna()
bins = [0, 25, 40, np.inf]
labels = ['vix_lo', 'vix_mid', 'vix_hi']
# Use left-closed intervals (left=True, so intervals are [a, b))
vix_regime = pd.cut(
    vix,
    bins=bins,
    labels=labels,
    right=False,  # left-closed: [a, b)
    include_lowest=True  # include 0 in the first bin
)

In [30]:
px.line(vix, template='plotly_white')

In [ ]:
import pandas as pd
import plotly.graph_objects as go

def add_regime_shading(fig: go.Figure, regime_series: pd.Series) -> go.Figure:
    """
    Adds shaded background rectangles to a Plotly figure based on a regime series.

    Parameters:
        fig (go.Figure): A Plotly figure (typically from plotly.express).
        regime_series (pd.Series): Series indexed by date with categorical values
                                   like 'vix_lo', 'vix_mid', 'vix_hi'.

    Returns:
        go.Figure: Modified figure with background shading.
    """
    # Define colors for each regime
    regime_colors = {
        'vix_lo': 'rgba(200, 255, 200, 0.2)',
        'vix_mid': 'rgba(255, 255, 150, 0.2)',
        'vix_hi': 'rgba(255, 150, 150, 0.2)'
    }

    # Create DataFrame for grouping contiguous blocks
    regime_df = vix_regime.to_frame(name='regime')
    regime_df['group'] = (regime_df['regime'] != regime_df['regime'].shift()).cumsum()
    
    # Add background rectangles for each regime block
    for (group, regime), block in regime_df.groupby(['group', 'regime']):
        start_date = block.index[0]
        end_date = block.index[-1]
        color = regime_colors.get(regime, 'rgba(200,200,200,0.1)')
        fig.add_shape(
            type='rect',
            xref='x',
            yref='paper',
            x0=start_date,
            x1=end_date,
            y0=0,
            y1=1,
            fillcolor=color,
            line_width=0,
            layer='below'
        )
    
    return fig


In [ ]:
# CHATGPT

import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
from typing import Optional, Dict

def add_regime_shading(
    fig: go.Figure,
    regime_series: pd.Series,
    regime_colors: Optional[Dict[str, str]] = None
) -> go.Figure:
    """
    Adds vertical shaded regions to a Plotly figure based on a regime time series.

    Parameters:
        fig (go.Figure): Existing Plotly figure (e.g., from plotly.express).
        regime_series (pd.Series): Series indexed by datetime with categorical regime labels.
        regime_colors (dict, optional): Optional mapping of regime name to RGBA color string.

    Returns:
        go.Figure: The figure with shaded regime regions added.
    """
    # Drop NaNs to avoid gaps or errors
    regime_series = regime_series.dropna()

    # Convert to DataFrame
    regime_df = regime_series.to_frame(name='regime')

    # Group by contiguous blocks of same regime
    regime_df['group'] = (regime_df['regime'] != regime_df['regime'].shift()).cumsum()

    # Determine color mapping if not provided
    unique_regimes = regime_df['regime'].unique()
    if regime_colors is None:
        default_palette = pc.qualitative.Set3
        rgba_template = 'rgba({}, {}, {}, 0.2)'
        regime_colors = {
            str(reg): rgba_template.format(*pc.hex_to_rgb(default_palette[i % len(default_palette)]))
            for i, reg in enumerate(unique_regimes)
        }

    # Add vertical rectangles
    for (group, regime), block in regime_df.groupby(['group', 'regime']):
        start_date = block.index[0]
        end_date = block.index[-1]
        color = regime_colors.get(regime, 'rgba(200,200,200,0.1)')
        fig.add_vrect(
            x0=start_date,
            x1=end_date,
            fillcolor=color,
            opacity=1.0,
            layer='below',
            line_width=0
        )

    return fig


In [43]:
vix_regime.to_frame(name='regime')

,regime
date,
2006-07-18,vix_lo
2006-07-19,vix_lo
2006-07-20,vix_lo
2006-07-21,vix_lo
2006-07-24,vix_lo
...,...
2025-05-16,vix_lo
2025-05-19,vix_lo
2025-05-20,vix_lo


In [36]:
regime_df = pd.DataFrame({'regime': regime_series})
regime_df

,regime
date,
2006-07-18,vix_lo
2006-07-19,vix_lo
2006-07-20,vix_lo
2006-07-21,vix_lo
2006-07-24,vix_lo
...,...
2025-05-16,vix_lo
2025-05-19,vix_lo
2025-05-20,vix_lo


In [ ]:
regimes = vix_regime
regime_df = pd.DataFrame({'regime': regimes})
regime_df['group'] = (regime_df['regime'] != regime_df['regime'].shift()).cumsum()
regime_df

,regime,group
date,,
2006-07-18,vix_lo,1
2006-07-19,vix_lo,1
2006-07-20,vix_lo,1
2006-07-21,vix_lo,1
2006-07-24,vix_lo,1
...,...,...
2025-05-16,vix_lo,171
2025-05-19,vix_lo,171
2025-05-20,vix_lo,171


In [ ]:
fig = px.line(vix, template='plotly_white')
regimes = vix_regime

add_regime_shading(fig, regimes)

C:\Users\bkrai\AppData\Local\Temp\ipykernel_13720\1051991829.py:28: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
from typing import Optional

def _generate_color_map(
    regime_series: pd.Series, 
    custom_map: Optional[dict] = None,
    default_alpha: float = 0.2
) -> dict:
    """Generate or finalize a color map for regimes.

    If `custom_map` is given, it's used as the base, and a 'default'
    color key is ensured for any unmapped values. Otherwise, colors are
    automatically generated for unique non-NaN regimes in `regime_series`
    using a Plotly qualitative palette. NaN values in the series will
    typically use the 'default' color unless `np.nan` is a key in
    `custom_map`.

    Parameters
    ----------
    regime_series : pd.Series
        The pandas Series containing regime values. Its `dtype` can be any
        hashable type (e.g., str, int, float, tuple) suitable for
        dictionary keys. `np.nan` is also handled if present.
    custom_map : dict, optional
        A user-defined dictionary mapping regime values to color strings
        (e.g., RGBA format like 'rgba(255,0,0,0.5)'). If None (default),
        colors are auto-generated.
    default_alpha : float, default 0.2
        The alpha (transparency) value to use for auto-generated colors
        or for the default fallback color if not fully specified in
        `custom_map`.

    Returns
    -------
    dict
        A dictionary mapping regime values (including a 'default' key) to
        RGBA color strings.
    """
    color_map = {}

    if custom_map:
        color_map = custom_map.copy()
        if 'default' not in color_map:
            color_map['default'] = f'rgba(128, 128, 128, {default_alpha})'
    else:
        defined_regimes = regime_series.dropna().unique()
        palette = pcolors.qualitative.Plotly
        
        for i, regime_val in enumerate(defined_regimes):
            hex_color = palette[i % len(palette)]
            rgb_tuple = pcolors.hex_to_rgb(hex_color)
            color_map[regime_val] = f'rgba({rgb_tuple[0]},{rgb_tuple[1]},{rgb_tuple[2]},{default_alpha})'
        
        default_hex = "#CCCCCC"
        default_rgb = pcolors.hex_to_rgb(default_hex)
        color_map['default'] = f'rgba({default_rgb[0]},{default_rgb[1]},{default_rgb[2]},{default_alpha})'
    
    return color_map


def get_color_map(categories: pd.Series) -> dict:
    """
    Generate a color map for unique categories in a pandas Series.

    Parameters
    ----------
    categories : pd.Series
        A pandas Series containing categorical values.

    Returns
    -------
    dict
        A dictionary mapping each unique category to a color string.
    """
    category_list = categories.dropna().unique()
    # palette = pc.qualitative.Plotly
    palette = pc.qualitative.Set3
    color_map = {cat: palette[i % len(palette)] for i, cat in enumerate(category_list)}
    # rgba_template = 'rgba({}, {}, {}, 0.2)'
    # color_map = {str(cat): rgba_template.format(*pc.hex_to_rgb(palette[i % len(palette)]))
    #                  for i, cat in enumerate(category_list)}

    return color_map


def add_regime_shading(
    fig: go.Figure, 
    regimes: pd.Series, 
    color_map: Optional[dict] = None
) -> go.Figure:
    """
    Adds shaded regions to a Plotly figure based on a regime time series.

    Parameters
    ----------
    fig : plotly.graph_objects.Figure
        The Plotly Figure object to be modified.
    regimes : pd.Series
        Series indexed by datetime with categorical regime labels.
    custom_color_map : dict, optional
        Optional mapping of regime name (or np.nan) to RGBA color string.
        If None, mapping is auto-generated by `_generate_regime_color_map`.

    Returns
    -------
    plotly.graph_objects.Figure
        The Plotly Figure modified with background shading.

    Notes
    -----
    - Shading is only applied for non-NaN regime blocks.

    """
    # TODO: Adjust the x1 parameter (currently using end_date directly) 
    #       in add_vrect to properly cover the full period of end_date
    #       (e.g., by using an offset like next business day, month end, or inferred frequency).
    #       Using x1=end_date may not shade the end_date's interval fully 
    #       and makes single-point blocks invisible.
    if regimes.empty:
        return fig

    if color_map is None:
        color_map = get_color_map(regimes)
    default_color = f'rgba(128, 128, 128, 0.2)'
    
    regime_blocks = (regimes
                     .ffill()
                     .to_frame(name='regime')
                     .assign(group=(lambda df: (df['regime'] != df['regime'].shift()).cumsum()))
                     )
    
    for (group, regime), block in regime_blocks.groupby(['group', 'regime']):
        if pd.isna(regime):
            continue
        start_date = block.index[0]
        end_date = block.index[-1]
        color = color_map.get(regime, default_color)
        fig.add_vrect(
            x0=start_date,
            x1=end_date,
            fillcolor=color,
            # opacity=1.0.
            layer='below',
            line_width=0
        )
        
    return fig

In [91]:
get_factor_data(read_cache=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Building 7 composite factors
Building 5 portfolio factors
Building portfolios: odict_keys(['Client', 'Vol_wtd_ptfl', 'EW_ptfl', 'Tracking_ptfl', 'Tracking_ptfl_penalty'])
Running portfolio Client with 2 out of 69 factors for 389 rebalancing dates


c:\Users\bkrai\Source\risk\risk_portfolios.py:346: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



Running portfolio Vol_wtd_ptfl with 17 out of 70 factors for 389 rebalancing dates
Running portfolio EW_ptfl with 9 out of 71 factors for 389 rebalancing dates
Running portfolio Tracking_ptfl with 9 out of 72 factors for 389 rebalancing dates
Running portfolio Tracking_ptfl_penalty with 9 out of 73 factors for 389 rebalancing dates
Calculating factor statistics
Factor construction complete


c:\Users\bkrai\Source\venv-bklm\Lib\site-packages\zarr\codecs\vlen_utf8.py:44: UserWarning:

The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.

c:\Users\bkrai\Source\venv-bklm\Lib\site-packages\zarr\codecs\vlen_utf8.py:44: UserWarning:

The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.

c:\Users\bkrai\Source\venv-bklm\Lib\site-packages\zarr\codecs\vlen_utf8.py:44: UserWarning:

The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.

c:\Users\bkrai\Source\venv-bklm\Lib\site-packages\zarr\codecs\vlen_utf8.py:44: UserWarning:

The codec `vlen-utf8` is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations

<xarray.Dataset> Size: 2GB
Dimensions:            (date: 8135, factor_name: 74, vol_type: 5,
                        factor_name_1: 74, corr_type: 5, portfolio_name: 5,
                        ticker: 21)
Coordinates:
  * date               (date) datetime64[ns] 65kB 1993-01-29 ... 2025-05-22
  * factor_name        (factor_name) object 592B 'SPY' ... 'Tracking_ptfl_pen...
  * vol_type           (vol_type) int64 40B 21 63 126 252 512
  * factor_name_1      (factor_name_1) object 592B 'SPY' ... 'Tracking_ptfl_p...
  * corr_type          (corr_type) int64 40B 21 63 126 252 512
  * portfolio_name     (portfolio_name) object 40B 'Client' ... 'Vol_wtd_ptfl'
  * ticker             (ticker) object 168B 'AGG' 'CMBS' ... 'VMBS' '^N225'
Data variables:
    ret                (date, factor_name) float64 5MB nan nan ... 27.42 27.16
    cret               (date, factor_name) float64 5MB nan nan ... 100.0 100.0
    vol                (vol_type, date, factor_name) float64 24MB nan ... 5.948
    corr               (corr_type, date, factor_name, factor_name_1) float64 2GB ...
    portfolio_weights  (date, portfolio_name, ticker) float64 7MB nan ... nan

In [89]:
vix_regime

date
2006-07-18    vix_lo
2006-07-19    vix_lo
2006-07-20    vix_lo
2006-07-21    vix_lo
2006-07-24    vix_lo
               ...  
2025-05-16    vix_lo
2025-05-19    vix_lo
2025-05-20    vix_lo
2025-05-21    vix_lo
2025-05-22    vix_lo
Name: cret, Length: 4743, dtype: category
Categories (3, object): ['vix_lo' < 'vix_mid' < 'vix_hi']

In [71]:
def get_vix_regime(cret):
    vix = factor_data.cret.sel(factor_name='^VIX3M').to_pandas().dropna()
    bins = [0, 25, 40, np.inf]
    labels = ['vix_lo', 'vix_mid', 'vix_hi']
    # Use left-closed intervals (left=True, so intervals are [a, b))
    return pd.cut(
        vix,
        bins=bins,
        labels=labels,
        right=False,  # left-closed: [a, b)
        include_lowest=True  # include 0 in the first bin
    )
    
vix_regime = get_vix_regime(factor_data.cret)
type(vix_regime)

pandas.core.series.Series

In [87]:
vix_regime, vix_bins = pd.qcut(vix.dropna().ffill(), [0, 0.15, 0.85, 1], labels=['vix_lo', 'vix_mid', 'vix_hi'], retbins=True)
vix_bins

array([11.02999973, 14.82999992, 27.9869997 , 72.98000336])

In [88]:
vix = factor_data.cret.sel(factor_name='^VIX3M').to_pandas().dropna()
bins = [0, 25, 40, np.inf]
labels = ['vix_lo', 'vix_mid', 'vix_hi']
# Use left-closed intervals (left=True, so intervals are [a, b))
vix_regime = pd.cut(
    vix,
    bins=bins,
    labels=labels,
    right=False,  # left-closed: [a, b)
    include_lowest=True  # include 0 in the first bin
)

VIX_COLORS = {
    'vix_lo':  'rgba(200, 255, 200, 0.2)',
    'vix_mid': 'rgba(255, 255, 150, 0.2)',
    'vix_hi':  'rgba(255, 150, 150, 0.2)'
}

fig = px.line(vix, template='plotly_white')
fig = add_regime_shading(fig, vix_regime, VIX_COLORS)
fig.show()

C:\Users\bkrai\AppData\Local\Temp\ipykernel_13720\589468117.py:134: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

